In [1]:
pip install azure-cognitiveservices-language-luis

Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
import time

# <Dependencies>
from azure.cognitiveservices.language.luis.authoring import LUISAuthoringClient
from azure.cognitiveservices.language.luis.runtime import LUISRuntimeClient
from msrest.authentication import CognitiveServicesCredentials

def quickstart():
    # <VariablesYouChange>
    authoringKey = 'xxxx'
    authoringEndpoint = 'https://luis0701test-authoring.cognitiveservices.azure.com/'
    predictionKey = 'xxxx'
    predictionEndpoint = 'https://luis0701test.cognitiveservices.azure.com/'

    # <VariablesYouDontNeedToChangeChange>
    appName = "Contoso Pizza Company"
    versionId = "0.1"
    intentName = "OrderPizzaIntent"

    # <AuthoringCreateClient>
    client = LUISAuthoringClient(authoringEndpoint, CognitiveServicesCredentials(authoringKey))

    # Create app関数を使用
    app_id = create_app(client, appName, versionId)

    # Intentを追加する
    client.model.add_intent(app_id, versionId, intentName)

    # Entity追加関数を使用
    add_entities(client, app_id, versionId)

    # Add labeled examples
    add_labeled_examples(client, app_id, versionId, intentName)

    # TrainAppVersion
    client.train.train_version(app_id, versionId)
    waiting = True
    while waiting:
        info = client.train.get_status(app_id, versionId)

        # get_status returns a list of training statuses, one for each model. Loop through them and make sure all are
        # done.
        waiting = any(map(lambda x: 'Queued' == x.details.status or 'InProgress' == x.details.status, info))
        if waiting:
            print("Waiting 10 seconds for training to complete...")
            time.sleep(10)
        else:
            print("trained")
            waiting = False

    # PublishVersion
    client.apps.update_settings(app_id, is_public=True)

    responseEndpointInfo = client.apps.publish(app_id, versionId, is_staging=False)

    # PredictionCreateClient
    runtimeCredentials = CognitiveServicesCredentials(predictionKey)
    clientRuntime = LUISRuntimeClient(endpoint=predictionEndpoint, credentials=runtimeCredentials)


    # QueryPredictionEndpoint
    # Production == slot name
    predictionRequest = {"query": "I want two small pepperoni pizzas with more salsa"}

    predictionResponse = clientRuntime.prediction.get_slot_prediction(app_id, "Production", predictionRequest)
    print("Top intent: {}".format(predictionResponse.prediction.top_intent))
    print("Sentiment: {}".format(predictionResponse.prediction.sentiment))
    print("Intents: ")

    for intent in predictionResponse.prediction.intents:
        print("\t{}".format(json.dumps(intent)))
    print("Entities: {}".format(predictionResponse.prediction.entities))


# アプリを作成して、app_idを返す関数
def create_app(client, appName, versionId):
    appDefinition = {
        "name": appName,
        "initial_version_id": versionId,
        "culture": "en-us"
    }

    # create app
    app_id = client.apps.add(appDefinition)

    # get app id - necessary for all other changes
    print("Created LUIS app with ID {}".format(app_id))
    return app_id


def add_entities(client, app_id, versionId):
    # Add Prebuilt entity
    client.model.add_prebuilt(app_id, versionId, prebuilt_extractor_names=["number"])

    # define machine-learned entity
    mlEntityDefinition = [
        {
            "name": "Pizza",
            "children": [
                {"name": "Quantity"},
                {"name": "Type"},
                {"name": "Size"}
            ]
        },
        {
            "name": "Toppings",
            "children": [
                {"name": "Type"},
                {"name": "Quantity"}
            ]
        }]

    # add entity to app
    modelId = client.model.add_entity(app_id, versionId, name="Pizza order", children=mlEntityDefinition)

    # 重要なフレーズを登録
    phraseList = {
        "enabledForAllModels": False,
        "isExchangeable": True,
        "name": "QuantityPhraselist",
        "phrases": "few,more,extra"
    }

    # add phrase list to app
    client.features.add_phrase_list(app_id, versionId, phraseList)

    # Get entity and subentities
    modelObject = client.model.get_entity(app_id, versionId, modelId)
    # grandchild_id関数を使用
    toppingQuantityId = get_grandchild_id(modelObject, "Toppings", "Quantity")
    pizzaQuantityId = get_grandchild_id(modelObject, "Pizza", "Quantity")

    # PizzaのQuantityにnumberエンティティを割り当てる
    prebuiltFeatureRequiredDefinition = {"model_name": "number", "is_required": True}
    client.features.add_entity_feature(app_id, versionId, pizzaQuantityId, prebuiltFeatureRequiredDefinition)

    # ToppingのQuantityにnumberエンティティを割り当てる
    prebuiltFeatureNotRequiredDefinition = {"model_name": "number"}
    client.features.add_entity_feature(app_id, versionId, toppingQuantityId, prebuiltFeatureNotRequiredDefinition)

    # フレーズリスト(QuantityOhraselistをToppingのQuantityに追加)
    phraseListFeatureDefinition = {"feature_name": "QuantityPhraselist", "model_name": None}
    client.features.add_entity_feature(app_id, versionId, toppingQuantityId, phraseListFeatureDefinition)

def add_labeled_examples(client, app_id, versionId, intentName):
    # Define labeled example
    labeledExampleUtteranceWithMLEntity = {
        "text": "I want two small seafood pizzas with extra cheese.",
        "intentName": intentName,
        "entityLabels": [
            {
                "startCharIndex": 7,
                "endCharIndex": 48,
                "entityName": "Pizza order",
                "children": [
                    {
                        "startCharIndex": 7,
                        "endCharIndex": 30,
                        "entityName": "Pizza",
                        "children": [
                            {
                                "startCharIndex": 7,
                                "endCharIndex": 9,
                                "entityName": "Quantity"
                            },
                            {
                                "startCharIndex": 11,
                                "endCharIndex": 15,
                                "entityName": "Size"
                            },
                            {
                                "startCharIndex": 17,
                                "endCharIndex": 23,
                                "entityName": "Type"
                            }]
                    },
                    {
                        "startCharIndex": 37,
                        "endCharIndex": 48,
                        "entityName": "Toppings",
                        "children": [
                            {
                                "startCharIndex": 37,
                                "endCharIndex": 41,
                                "entityName": "Quantity"
                            },
                            {
                                "startCharIndex": 43,
                                "endCharIndex": 48,
                                "entityName": "Type"
                            }]
                    }
                ]
            }
        ]
    }

    print("Labeled Example Utterance:", labeledExampleUtteranceWithMLEntity)

    # Add an example for the entity.
    # Enable nested children to allow using multiple models with the same name.
    # The quantity subentity and the phraselist could have the same exact name if this is set to True
    client.examples.add(app_id, versionId, labeledExampleUtteranceWithMLEntity, {"enableNestedChildren": True})

def get_grandchild_id(model, childName, grandChildName):
    theseChildren = next(filter((lambda child: child.name == childName), model.children))
    theseGrandchildren = next(filter((lambda child: child.name == grandChildName), theseChildren.children))

    grandChildId = theseGrandchildren.id

    return grandChildId


# </AuthoringSortModelObject>

quickstart()

Created LUIS app with ID 72c86250-5a96-474b-a806-c5c94dbf6583
Labeled Example Utterance: {'text': 'I want two small seafood pizzas with extra cheese.', 'intentName': 'OrderPizzaIntent', 'entityLabels': [{'startCharIndex': 7, 'endCharIndex': 48, 'entityName': 'Pizza order', 'children': [{'startCharIndex': 7, 'endCharIndex': 30, 'entityName': 'Pizza', 'children': [{'startCharIndex': 7, 'endCharIndex': 9, 'entityName': 'Quantity'}, {'startCharIndex': 11, 'endCharIndex': 15, 'entityName': 'Size'}, {'startCharIndex': 17, 'endCharIndex': 23, 'entityName': 'Type'}]}, {'startCharIndex': 37, 'endCharIndex': 48, 'entityName': 'Toppings', 'children': [{'startCharIndex': 37, 'endCharIndex': 41, 'entityName': 'Quantity'}, {'startCharIndex': 43, 'endCharIndex': 48, 'entityName': 'Type'}]}]}]}
Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
trained
Top intent: OrderPizzaIntent
Sentiment: None
Intents: 
	"OrderPizzaIntent"
Entities: {'Pizza order': [{'Pizz